In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models
import pandas as pd
from PIL import Image
import os

In [71]:
# Definér en brugerdefineret datasetklasse til at indlæse billeder og labels
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 0] + '.jpg'  # Antager at billederne er gemt som JPG-filer
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path)
        label = self.data.iloc[idx, 1:-1].values.astype('float32')  # Labels for features
        if self.transform:
            image = self.transform(image)
        return image, label

In [72]:
# Definér transformations for billederne
transform = transforms.Compose([
    transforms.Resize((135, 135)),  # Tilpas til størrelsen på billeder, som mange CNN-modeller forventer
    transforms.ToTensor()
])

In [73]:
# Opret en instans af din brugerdefinerede datasetklasse til træningsdatasættet
train_dataset = CustomDataset(csv_file='pawpularity/train.csv', root_dir='pawpularity/train/', transform=transform)

In [74]:
# Opret en dataloader for træningsdatasættet
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [75]:
# Definér CNN-arkitekturen
class CNN(nn.Module):
    def __init__(self, num_features):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.fc1 = nn.Linear(64 * 5 * 5, 128) # Efter at have max-poolet 3 gange, så ender vi med billeder der svarer til 5px * 5px
        self.fc2 = nn.Linear(128, num_features)  # Outputlaget har antallet af features som output

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [76]:
# Opret en instans af CNN-modellen
model = CNN(num_features=12)  # Antagelse: Der er 12 features eksklusive Pawpularity

In [77]:
# Definér tab-funktion og optimeringsalgoritme
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [78]:
num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')


Epoch [1/20], Loss: 6.0957
Epoch [2/20], Loss: 5.9215
Epoch [3/20], Loss: 5.8768
Epoch [4/20], Loss: 5.8439
Epoch [5/20], Loss: 5.8076
Epoch [6/20], Loss: 5.7685
Epoch [7/20], Loss: 5.7308
Epoch [8/20], Loss: 5.6959
Epoch [9/20], Loss: 5.6450
Epoch [10/20], Loss: 5.6051
Epoch [11/20], Loss: 5.5555
Epoch [12/20], Loss: 5.4952
Epoch [13/20], Loss: 5.4421
Epoch [14/20], Loss: 5.3856
Epoch [15/20], Loss: 5.3236
Epoch [16/20], Loss: 5.2618
Epoch [17/20], Loss: 5.1986
Epoch [18/20], Loss: 5.1238
Epoch [19/20], Loss: 5.0625
Epoch [20/20], Loss: 4.9913


In [79]:
# Gem den trænede model
torch.save(model.state_dict(), 'trained_model3.pth')